In [1]:
import json

with open('../input/train.json', 'r') as file:
    # 从文件中加载数据
    data = json.load(file)
print(len(data))
train_5k = data[:5000]
with open('../input/train_5k.json', 'w') as outfile:
    json.dump(train_5k, outfile, ensure_ascii=False)

44049


In [4]:
"""
分割文件

"""

# 打开 JSON 文件
with open('../input/test_full.json', 'r') as file:
    # 从文件中加载数据
    data = json.load(file)

# 提取前 3000 条数据
eval_data = data[:3000]

# 将提取的数据保存到新的 JSON 文件中
with open('../input/eval.json', 'w') as outfile:
    json.dump(eval_data, outfile, ensure_ascii=False)


test_data = data[3000:]

# 将提取的数据保存到新的 JSON 文件中
with open('../input/test.json', 'w') as outfile:
    json.dump(test_data, outfile, ensure_ascii=False)



In [3]:
"""
查账标注标签

"""

with open('../input/eval.json', 'r') as file:
    # 从文件中加载数据
    data = json.load(file)

labels = ['B-incomplete', 'E-incomplete', 'S-district']

for i in range(len(data)):
  for token in data[i]['tokens']:
    if token[1] in labels:
      print(data[i])

print('done')


{'text': '山东省单谢集乡袁新庄行政村大朱庄0000号', 'tokens': [['山', 'B-province'], ['东', 'I-province'], ['省', 'E-province'], ['单', 'S-district'], ['谢', 'B-poiCollection'], ['集', 'I-poiCollection'], ['乡', 'E-poiCollection'], ['袁', 'B-village'], ['新', 'I-village'], ['庄', 'I-village'], ['行', 'I-village'], ['政', 'I-village'], ['村', 'E-village'], ['大', 'B-poiCollection'], ['朱', 'I-poiCollection'], ['庄', 'E-poiCollection'], ['0', 'B-building'], ['0', 'I-building'], ['0', 'I-building'], ['0', 'I-building'], ['号', 'E-building']]}
done


In [4]:
"""
删除指定标注带标签的数据

"""

with open('../input/eval.json', 'r') as file:
    # 从文件中加载数据
    data = json.load(file)

labels = ['B-incomplete', 'E-incomplete', 'S-district']
new_datas = []
for i in range(len(data)):
  new_datas.append(data[i])
  for token in data[i]['tokens']:
    if token[1] in labels:
      new_datas.pop()
      break

with open('../input/eval_clean.json', 'w') as outfile:
    json.dump(new_datas, outfile, ensure_ascii=False)
    
print('done')


done


In [2]:
import os
def generate_output_file(input_file):
  # 获取文件路径和文件名
  file_path, file_name = os.path.split(input_file)
  # 获取文件名和扩展名
  file_name_no_ext, extension = os.path.splitext(file_name)
  # 组合新文件名
  output_file = f"{file_name_no_ext}_ok{extension}"
  # 组合新文件路径
  output_file_path = os.path.join(file_path, output_file)
  return output_file_path

In [3]:
"""
删除非法长度的数据
1. text和label不一致
2. text长度超过50 or label长度超过50
3. 有不认识的label

"""

import json

files = ['../input/linan/train-config_linan_住房-业务-2k-first-0311.json', '../input/linan/验证格式-住房-业务-2k-second-0311.json', '../input/test.json']
unknow_labels = ['B-incomplete', 'E-incomplete', 'S-district']

for file in files:
  error1 = 0
  error2 = 0
  error3 = 0
  print(f'handle: {file}')

  with open(file, 'r') as f:
    data = json.load(f)

  new_data = []
  for i in range(len(data)):
    token_length = len(data[i]["tokens"])
    text_length = len(data[i]["text"])

    if token_length != text_length:
      # print(f'length not equal: {data[i]["text"]} {data[i]["tokens"]}')
      error1+=1
      continue
    
    if token_length > 50 or text_length > 50:
      # print(f'length not equal: {data[i]["text"]} {data[i]["tokens"]}')
      error2+=1
      continue
    
    found_unknow_label = False
    for token in data[i]['tokens']:
      if token[1] in unknow_labels:
        # print(f'unknow label: {data[i]["text"]} {data[i]["tokens"]}')
        found_unknow_label = True
        break

    if found_unknow_label:
      error3+=1
      continue

    new_data.append(data[i])

  print(f'all: {len(data)}, success: {len(new_data)}, error1: {error1}, error2: {error2}, error3: {error3}')

  out_file = generate_output_file(file)
  with open(out_file, 'w') as f:
    json.dump(new_data, f, ensure_ascii=False)

print('done')

handle: ../input/linan/train-config_linan_住房-业务-2k-first-0311.json
all: 80191, success: 79140, error1: 0, error2: 1051, error3: 0
handle: ../input/linan/验证格式-住房-业务-2k-second-0311.json
all: 1999, success: 1999, error1: 0, error2: 0, error3: 0
handle: ../input/test.json
all: 5330, success: 5330, error1: 0, error2: 0, error3: 0
done
